# TRPO

## 一、简介

之前介绍的基于策略的方法包括策略梯度算法和Actor-Critic 算法。

这些算法虽然简单、直观，但在实际应用过程中会遇到训练不稳定的情况。

回顾一下基于策略的方法：参数化智能体的策略，并设计衡量策略好坏的目标函数，
通过梯度上升的方法来最大化这个目标函数，使得策略最优。

具体来说，假设策略为$\pi_\theta(a|s)$，参数为$\theta$，目标函数为:

$J(\theta)=E_{s_0}[V^{\pi_\theta}(s_0)]=E_{\pi_\theta}[\Sigma_{t=0}\gamma^tr(s_t,a_t)]$，

基于策略的方法目标：找到$\theta^*=argmax_\theta J(\theta)$

但是这种算法有一个明显的缺点：当策略网络是深度模型时，
沿着策略梯度更新参数，很有可能由于步长太长，策略突然显著变差，进而影响训练效果

针对以上问题，我们考虑在更新时找到一块信任区域（trust region），
在这个区域上更新策略时能够得到某种策略性能的安全性保证，
这就是信任区域策略优化
（trust region policy optimization，TRPO）
算法的主要思想。

TRPO 算法在 2015 年被提出，它在理论上能够保证策略学习的性能单调性，
并在实际应用中取得了比策略梯度算法更好的效果。

策略目标（这里懒得推了）：
